<a href="https://colab.research.google.com/github/jychen74/notes/blob/main/nanopore_%E5%88%86%E6%9E%90%E4%BD%BF%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 環境建置與打包 (build and package)



1.   掛載 googld drive
2.   下載及安裝 micromamba
3.   安裝分析軟體
4.   驗證工具是否可執
5.   打包備份到Google drive

------

## 日常載入及使用

> 載入 Nanopore 分析環境 (在其他 colab notebook 複製這段過去)   
  **記得要改版本號，不然是舊版**




------


In [ ]:
# @title 1. 掛載 Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

# 設定存放環境包的路徑 (在Drive建Colab_Envs資料夾)
# 如果資料夾不存在，我們會自動建立
EXPORT_DIR = '/content/drive/MyDrive/Colab_Notebooks/Colab_Envs'
if not os.path.exists(EXPORT_DIR):
    os.makedirs(EXPORT_DIR)
    print(f"已建立資料夾: {EXPORT_DIR}")
else:
    print(f"將儲存至: {EXPORT_DIR}")

In [ ]:
# @title 2下載並安裝 Micromamba
# 下載 micromamba 二進位檔
!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba

# 設定環境變數與初始化
import os
os.environ['MAMBA_ROOT_PREFIX'] = '/content/micromamba'
!bin/micromamba shell init -s bash -p /content/micromamba

# 驗證安裝
!bin/micromamba --version

In [ ]:
# @title 3. 建立 Nanopore 分析環境並安裝 Nanopore 分析工具
# 定義環境路徑 (勿更改，保持固定路徑以確保相容性)
ENV_PREFIX = "/content/nano_env"

# 開始安裝
# -c bioconda -c conda-forge: 指定生物資訊常用的channel
# minimap2 kraken2 samtools nanoplot: 您需要的軟體
print("正在安裝環境，這可能需要 3-5 分鐘，請稍候...")

!bin/micromamba create -y -p {ENV_PREFIX} \
    -c bioconda -c conda-forge \
    minimap2 \
    kraken2 \
    samtools \
    nanoplot \
    nanofilt \
    flye \
    pigz

# 清理暫存檔以縮小體積
!bin/micromamba clean --all --yes

In [ ]:
# @title 4. 驗證工具是否可執行
import os
# 暫時將新環境加入 PATH 進行測試
os.environ['PATH'] = f"/content/nano_env/bin:{os.environ['PATH']}"

print("Testing Minimap2 version:")
!minimap2 --version

print("\nTesting Kraken2 version:")
!kraken2 --version

In [ ]:
# @title 5. 打包並備份到 Google Drive
import time

# 設定壓縮檔名稱 (建議加上日期以便版本管理)
from datetime import datetime
date_str = datetime.now().strftime("%Y%m%d")
TAR_NAME = f"nanopore_env_{date_str}.tar.gz"
SAVE_PATH = os.path.join(EXPORT_DIR, TAR_NAME)

print(f"開始打包環境至 {SAVE_PATH} ...")
start_time = time.time()

# 使用 tar 打包，並透過 pigz 進行多核心壓縮
# c: create, v: verbose, f: file
# --use-compress-program=pigz: 使用多核心加速
!tar --use-compress-program=pigz -cf {SAVE_PATH} -C /content nano_env

end_time = time.time()
print(f"打包完成！耗時: {end_time - start_time:.2f} 秒")
print(f"檔案已儲存於: {SAVE_PATH}")

## 日常載入及使用

In [ ]:
# @title 載入 Nanopore 分析環境 (在其他 colab notebook 複製這段過去)
import os
import time
from google.colab import drive

# 1. 掛載 Drive
drive.mount('/content/drive')

# 2. 設定環境包路徑 (請修改為您 Drive 中實際的檔案名稱)
# 這裡要手動改檔案名稱，例如 'nanopore_env_20260204.tar.gz'，確定是用最新版的！！
TAR_PATH = '/content/drive/MyDrive/Colab_Notebooks/Colab_Envs/nanopore_env_20260204.tar.gz'

# 3. 解壓縮
print("正在載入環境...大約 1 分鐘")
start_time = time.time()

# 確保目標目錄不存在 (避免重複解壓)
if not os.path.exists("/content/nano_env"):
    # 使用 pigz 快速解壓
    !tar --use-compress-program=pigz -xf {TAR_PATH} -C /content

    # 4. 設定 PATH
    # 這是最關鍵的一步，讓系統找得到您的工具
    os.environ['PATH'] = f"/content/nano_env/bin:{os.environ['PATH']}"

    print(f"環境載入完成！耗時: {time.time() - start_time:.2f} 秒")
else:
    print("環境似乎已經載入過了。")

# 5. 測試一下
print("目前使用的 Minimap2 位置:")
!which minimap2
!minimap2 --version